In [2]:
import json
import nltk
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
nltk.download('punkt')  # Cần tải dữ liệu cho tokenizer
from nltk.tokenize import word_tokenize
import regex as re
import random


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [18]:
import nltk.data
try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt tokenizer is available.")
except LookupError:
    print("Punkt tokenizer is not available. Downloading it...")
    nltk.download('punkt')
print(nltk.data.path)

Punkt tokenizer is available.
['C:\\Users\\ASUS/nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data', 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [5]:
with open('intents.json','r',encoding='utf-8') as file:
    intents = json.load(file)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Xin chào ad',
    'Chào admin',
    'Ad ơi',
    'hi',
    'hello',
    'có ai ở đây không ạ',
    'bắt đầu',
    'Hey!',
    'ad ơi em muốn tư vấn tour với ạ',
    'ai có thể giúp em không',
    'dạ cho em hỏi',
    'cho hỏi có ai chat được bây giờ không',
    'em cần tư vấn ạ',
    'làm ơn giúp em',
    'em đang cần một ít thông tin',
    'chào ad, em có một câu hỏi',
    'em có một vấn đề cần được giải thích'],
   'responses': ['Chào bạn, DANANG tourist chuyên về các tour của thành phố Đà Nẵng xin nghe.Mình có thể giúp gì cho bạn được ạ']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'goodbye',
    'good bye',
    'Tạm biệt',
    'Bai',
    'pp',
    'hẹn gặp lại',
    'bái bai ad nha',
    'cảm ơn và chào tạm biệt',
    'chúc một ngày tốt lành và tạm biệt',
    'em sẽ liên hệ lại sau ạ',
    'em sẽ quay lại sau',
    'em sẽ tìm đến ad sau',
    'em sẽ quay lại nếu có thêm câu hỏi'],
   'responses': ['Chúc bạn ngày mới tốt đẹp',


In [14]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
def text_process(text):
    text = convert_unicode(text)
    text = text.lower()
    text = re.sub(r'[^swáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
    text = re.sub(r's+', ' ', text).strip()
    return text
def sen_to_vec(tokenized_sen,all_words):
    vec = np.zeros(len(all_words),dtype=np.float32)
    for index,w in enumerate(all_words):
        if w in tokenized_sen:
            vec[index] = 1.0
    return vec

In [30]:
all_words = []
tags = []
pat_tag = []
print(intents)
for intent in intents['intents']:
    tag = intent['tag']  # Lấy giá trị của tag
    patterns = intent['patterns']  # Lấy danh sách patterns tương ứng với tag
    print(f"Tag: {tag}")
    print("Patterns:")
    for pattern in patterns:
        print(f"  - {pattern}")
        pattern = text_process(pattern)
        if isinstance(pattern, str) and pattern:  # Đảm bảo pattern là chuỗi và không rỗng
            w = word_tokenize(pattern)  # Thực hiện tokenization
            all_words.extend(w)
            pat_tag.append((w, tag))
        else:
            print(f"Error: pattern is invalid (type: {type(pattern)}, value: {pattern})")
# for intent in intents['intents']:
#     tag = intent['tag']
#     tags.append(tag)
#     for pattern in intent['patterns']:
#         pattern = text_process(pattern)
#         w = word_tokenize(pattern)
#         all_words.extend(w)
#         pat_tag.append((w,tag))

# for intent in intents.get('intents', []):
#     tag = intent.get('tag', '')
#     tags.append(tag)
#     for pattern in intent.get('patterns', []):
#         pattern = text_process(pattern)
#         w = word_tokenize(pattern)  # Thực hiện tokenization
#         all_words.extend(w)
#         pat_tag.append((w, tag))

{'intents': [{'tag': 'greeting', 'patterns': ['Xin chào ad', 'Chào admin', 'Ad ơi', 'hi', 'hello', 'có ai ở đây không ạ', 'bắt đầu', 'Hey!', 'ad ơi em muốn tư vấn tour với ạ', 'ai có thể giúp em không', 'dạ cho em hỏi', 'cho hỏi có ai chat được bây giờ không', 'em cần tư vấn ạ', 'làm ơn giúp em', 'em đang cần một ít thông tin', 'chào ad, em có một câu hỏi', 'em có một vấn đề cần được giải thích'], 'responses': ['Chào bạn, DANANG tourist chuyên về các tour của thành phố Đà Nẵng xin nghe.Mình có thể giúp gì cho bạn được ạ']}, {'tag': 'goodbye', 'patterns': ['Bye', 'goodbye', 'good bye', 'Tạm biệt', 'Bai', 'pp', 'hẹn gặp lại', 'bái bai ad nha', 'cảm ơn và chào tạm biệt', 'chúc một ngày tốt lành và tạm biệt', 'em sẽ liên hệ lại sau ạ', 'em sẽ quay lại sau', 'em sẽ tìm đến ad sau', 'em sẽ quay lại nếu có thêm câu hỏi'], 'responses': ['Chúc bạn ngày mới tốt đẹp', 'Hẹn bạn một ngày tuyệt vời tại DANANG tourist ạ', 'Hy vọng bạn đã có câu trả lời tốt nhất cho mình , hẹn bạn một người đẹp trời t

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\ASUS/nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [32]:
import nltk
from nltk.tokenize import word_tokenize



all_words = []
tags = []
pat_tag = []


# Vòng lặp qua các intents
for intent in intents['intents']:
    tag = intent['tag']  # Lấy tag hiện tại
    tags.append(tag)
    for pattern in intent['patterns']:
        # Giả sử bạn có một hàm text_process để xử lý văn bản trước khi token hóa
        pattern = pattern.strip()  # Loại bỏ khoảng trắng đầu cuối (nếu cần)
        
        # Kiểm tra nếu pattern là chuỗi hợp lệ
        if isinstance(pattern, str) and pattern:
            w = word_tokenize(pattern)  # Thực hiện tokenization
            all_words.extend(w)  # Thêm các từ vào danh sách all_words
            pat_tag.append((w, tag))  # Ghép các từ với tag
        else:
            print(f"Error: pattern is invalid (type: {type(pattern)}, value: {pattern})")

# In kết quả ra để kiểm tra
print("All words:", all_words)
print("Patterns with tags:", pat_tag)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\ASUS/nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [33]:
ignore_char = ['?','!','.',',',"ad", "ạ", "ơi", "cho", "hỏi", "về", "làm", "thế", "nào", "có",
    "của", "cho", "ở", "và", "có", "không", "này", "khi", "để", "đi",
    "em", "mình", "bạn", "có", "gì", "nhé", "được", "ra", "sao", "mấy",
    "đó", "người", "như", "từ", "là", "cách", "nhưng", "theo", "hay",
    "với", "nên", "sẽ", "những", "làm", "nói", "được", "nếu", "đã"]
bag_word = [word for word in all_words if word not in ignore_char]
bag_word = sorted(set(bag_word))
tags = sorted(set(tags))

In [34]:
X_train = []
y_train = []

for (pattern_sen,tag) in pat_tag:
    vec = sen_to_vec(pattern_sen,bag_word)
    X_train.append(vec)

    label = tags.index(tag)
    y_train.append([label])

X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = to_categorical(y_train, len(tags))

In [35]:
model = Sequential()

model.add(Dense(len(bag_word), input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tags), activation='softmax'))

initial_learning_rate = 0.01

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [36]:
with open('state_vector.json','r') as file:
    states = json.load(file)


all_words_state = []
state_tags = []
pat_tag = []
for state in states['state_vector']:
    state_tag = state['tag']
    state_tags.append(state_tag)
    for pattern in state['patterns']:
        pattern = text_process(pattern)
        w = word_tokenize(pattern)
        all_words_state.extend(w)
        pat_tag.append((w,state_tag))


ignore_char = ['?','!','.',',','học']
bag_word_state = [word for word in all_words_state if word not in ignore_char]
bag_word_state = sorted(set(bag_word_state))
state_tags = sorted(set(state_tags))


X_train_state = []
y_train_state = []
for (pattern_sen,tag) in pat_tag:
    vec = sen_to_vec(pattern_sen,bag_word_state)
    X_train_state.append(vec)
    label = state_tags.index(tag)
    y_train_state.append([label])
X_train_state = np.array(X_train_state)
y_train_state = np.array(y_train_state)
y_train_state = to_categorical(y_train_state, len(state_tags))


model_state= Sequential()
model_state.add(Dense(len(bag_word_state), input_shape=(len(X_train_state[0]),), activation='relu'))
model_state.add(Dropout(0.5))
model_state.add(Dense(16, activation='relu'))
model_state.add(Dropout(0.5))
model_state.add(Dense(len(state_tags), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model_state.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model_state.fit(X_train_state, y_train_state, epochs=200, batch_size=5, verbose=1)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1884: character maps to <undefined>

In [37]:
def get_pred_intent(chat,bag_word):
    chat = text_process(chat)
    w = np.array(word_tokenize(chat))
    vec = sen_to_vec(w,bag_word).reshape(1,-1)
    pred = model.predict(vec)
    prob = pred[0, pred.argmax(axis=-1)[0]]
    tag = tags[pred.argmax(axis=-1)[0]]
    return prob,tag
def get_pred_state(chat,bag_word):
    chat = text_process(chat)
    w = np.array(word_tokenize(chat))
    vec = sen_to_vec(w,bag_word).reshape(1,-1)
    pred = model_state.predict(vec)
    prob_state = pred[0, pred.argmax(axis=-1)[0]]
    state = state_tags[pred.argmax(axis=-1)[0]]
    return prob_state, state

In [38]:
with open('database.json','r') as file:
    db = json.load(file)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 748: character maps to <undefined>

In [ ]:
db['tuition']['Bigdata & Machinelearning']
db['std_point']['Bigdata & Machinelearning']

In [39]:
bot_name = 'Bot'
print("-------Bắt đầu-------\n") Xin chào, mình là chatbot ảo có thể cung cấp cho bạn các thông tin \n liên quan đến tuyển sinh đại học như là: học phí, điểm chuẩn,... tại trường đại học Duy Tân"
print(f"{bot_name}:)
slots =[]
while 1:

    chat = text_process(input("User: "))
    if chat == "quit":
        print("Cảm ơn bạn đã sử dụng Chatbot!")
        break

    prob_intent, tag = get_pred_intent(chat,bag_word)
    print("1.",prob_intent,tag)
    prob_state, state = get_pred_state(chat,bag_word_state)
    print("2.",prob_state,state)
    if prob_state > 0.85:
        slots.clear()
        slots.insert(1,state)
    if tag != "unknown" and prob_intent > 0.85:
        slots.insert(0,tag)
    if slots[0] in ['tuition','std_point'] or slots[0] in state_tags:
        while (len(slots)!=2):
            if slots[0] in state_tags:#thieu intent
                while 1:
                    print(f"{bot_name}: Bạn cần hỏi thông tin nào của ngành {slots[0]}?")
                    chat = text_process(input("User: "))
                    if chat == "quit":
                        print("Cảm ơn bạn đã sử dụng Chatbot!")
                        exit(0)
                    prob_intent, tag = get_pred_intent(chat,bag_word)
                    print("1.",prob_intent, tag)
                    if prob_intent > 0.85:
                        slots.insert(0,tag)
                        break
            else:
                while 1: #thieu state doi tuong
                    print(f"{bot_name}: Bạn cần hỏi thông tin này của ngành nào?")
                    chat = text_process(input("User: "))
                    if chat == "quit":
                        print("Cảm ơn bạn đã sử dụng Chatbot!")
                        exit(0)
                    prob_state, state = get_pred_state(chat,bag_word_state)
                    print("2.",prob_state, state)
                    if prob_state > 0.85:
                        slots.insert(1,state)
                        break
        if slots[0] in ['tuition','std_point']:
            for intent in intents["intents"]:
                if tag == intent['tag']:
                    res = intent['responses'][0].format(slots[1],db[slots[0]][slots[1]])
                    print(f"{bot_name}: {res}")
                    break;
        else:
            for intent in intents["intents"]:
                if tag == intent['tag']:
                    print(f"{bot_name}: {random.choice(intent['responses'])}")
                    break;
        slots.pop(0)
    else:
        for intent in intents["intents"]:
            if tag == intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
                break;
        slots.clear()

SyntaxError: invalid syntax (2479776384.py, line 2)